<a href="https://colab.research.google.com/github/sugarforever/openai-swarm-tutorials/blob/main/openai_swarm_get_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Swarm Get Started

OpenAI Swarm is an educational framework exploring ergonomic, lightweight multi-agent orchestration. It's managed by OpenAI Solution team.

GitHub: https://github.com/openai/swarm

In this get-started tutorial, we will showcase how multi-agents are coordinated by Swarm, and walk you through the basic components:
- Swarm client
- Agents
- Functions
  - Basics
  - Schema

## What is AI Agent?

AI agents are autonomous intelligent systems performing specific tasks without human intervention.

## Overview

Let's take a quick overview with some examples on Swarm usage.

In [38]:
!pip install git+https://github.com/openai/swarm.git

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-akbqp876
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-akbqp876
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Set up OpenAI execution environment

Retrieve API key from Google Colab secrets and inject into enviroment variables

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### A Basic Example

Let's see how 2 agents are coordinated.

In [45]:
from swarm import Swarm, Agent

client = Swarm()

In [46]:
def transfer_to_agent_b():
    return agent_b


agent_a = Agent(
    name="Agent A",
    instructions="You are a helpful agent.",
    functions=[transfer_to_agent_b],
)

agent_b = Agent(
    name="Agent B",
    instructions="Only speak in Chinese.",
)

In [47]:
response = client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "I want to talk to agent B."}],
)

In [48]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_Oep2ErkZAlQyo0GVlJrWu8SM',
    'function': {'arguments': '{}', 'name': 'transfer_to_agent_b'},
    'type': 'function'}],
  'sender': 'Agent A'},
 {'role': 'tool',
  'tool_call_id': 'call_Oep2ErkZAlQyo0GVlJrWu8SM',
  'tool_name': 'transfer_to_agent_b',
  'content': '{"assistant": "Agent B"}'},
 {'content': '抱歉，我无法连接到另一个代理。如果有任何问题或需要帮助，请告诉我，我会尽力协助你。',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'sender': 'Agent B'}]

In [44]:
response.messages[-1]["content"]

'对不起，我不能直接将您转接到另一位助手，但您可以告诉我需要帮助的具体问题，我会尽力帮助您。'

### Example 2 - Weather Agent

It's based on one of the examples OpenAI Swarm published - https://github.com/openai/swarm/tree/main/examples/weather_agent

In [ ]:
import json

def get_weather(location, time="now"):
    """Get the current weather in a given location. Location MUST be a city."""
    return json.dumps({"location": location, "temperature": "65", "time": time})


def send_email(recipient, subject, body):
    print("Sending email...")
    print(f"To: {recipient}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")
    return "Sent!"


weather_agent = Agent(
    name="Weather Agent",
    instructions="You are a helpful agent.",
    functions=[get_weather, send_email],
)

In [49]:
response = client.run(
    agent=weather_agent,
    messages=[{"role": "user", "content": "Send me the weather in Beijing today."}],
)

In [50]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_yGlAzBk4xGqi3w5MIcQsPBSn',
    'function': {'arguments': '{"location":"Beijing"}', 'name': 'get_weather'},
    'type': 'function'}],
  'sender': 'Weather Agent'},
 {'role': 'tool',
  'tool_call_id': 'call_yGlAzBk4xGqi3w5MIcQsPBSn',
  'tool_name': 'get_weather',
  'content': '{"location": "Beijing", "temperature": "65", "time": "now"}'},
 {'content': 'The current temperature in Beijing is 65°F.',
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'sender': 'Weather Agent'}]

## Components Introduction

### Swarm Client

Swarm client internally just instantiates an OpenAI client.

In [51]:
from swarm import Swarm
from openai import OpenAI

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url="https://api.openai.com/v1"
)

swarm = Swarm(client=client)

### Agents

An Agent encapsulates a set of instructions with a set of functions (plus some additional settings below), and has the capability to hand off execution to another Agent.

In [54]:
def instructions(context_variables):
   user_name = context_variables["user_name"]
   return f"You are the receptionist of company Tesla. {user_name} is coming to your reception. You should welcome with its name."

agent = Agent(instructions=instructions)

Swarm client runs with the agent.

In [55]:
response = swarm.run(
    agent=agent,
    messages=[{"role":"user", "content": "Hi!"}],
    context_variables={
        "user_name":"VerySmallWoods"
    }
)
print(response.messages[-1]["content"])

Hello, VerySmallWoods! Welcome to Tesla. How can I assist you today?


### Functions

The functions called by Swarm Agents should usually return a str (values will be attempted to be cast as a str), or an Agent.

When an Agent is returned, the execution will be transferred to that one.

If a function defines a context_variables parameter, it will be populated by the context_variables passed into
`client.run()`.


#### Handsoff and Context Variables Update

An Agent can hand off to another Agent by returning it in a function, and update the context variables by returning a more complete `Result` object.

**Handsoff to another agent**

In [56]:
sales_agent = Agent(name="Sales Agent")

def transfer_to_sales():
   return sales_agent

agent = Agent(functions=[transfer_to_sales])

response = swarm.run(agent, [{"role":"user", "content":"Transfer me to sales."}])
print(response.agent.name)

Sales Agent


In [57]:
response.messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_XnGD4raVQZz9CqWHyO3QF7FP',
    'function': {'arguments': '{}', 'name': 'transfer_to_sales'},
    'type': 'function'}],
  'sender': 'Agent'},
 {'role': 'tool',
  'tool_call_id': 'call_XnGD4raVQZz9CqWHyO3QF7FP',
  'tool_name': 'transfer_to_sales',
  'content': '{"assistant": "Sales Agent"}'},
 {'content': "I've transferred you to the sales department. How can they assist you today?",
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': None,
  'sender': 'Sales Agent'}]

**Update the context variables**

In [58]:
sales_agent = Agent(name="Sales Agent")

def talk_to_sales():
   print("Hello, World!")
   return Result(
       value="Done",
       agent=sales_agent,
       context_variables={"department": "sales"}
   )

agent = Agent(functions=[talk_to_sales])

response = swarm.run(
   agent=agent,
   messages=[{"role": "user", "content": "Transfer me to sales"}],
   context_variables={"user_name": "John"}
)
print(response.agent.name)
print(response.context_variables)

Hello, World!


NameError: name 'Result' is not defined